# 3. Other data notebook
----------------
-----------

## Table of contents <a name="contentz"></a>
* [Generic location temperatures](#temps)
* [Generic property web page](#prop-page)
* [Property prices](#prop-price)
* [Property rents](#prop-rent)
* [City indices from Numbeo](#numb)
    * [Quality of life index](#qual-lif)
    * [Cost of living index](#cost-liv)
    * [Property stats](#prop-stats)
    * [Load consolidated table of indices](#import-numb)
* [Mapping with colors based on index](#colors)

---------
-------

Load libraries

In [2]:
%%time

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

# import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
# # from requests import get
# from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import folium # map rendering library

from bs4 import BeautifulSoup as bs
import branca.colormap as bcm

print('Libraries imported.')

Libraries imported.
Wall time: 6.62 s


---------
-------

### Scraping for city temperatures - Wikipedia <a name="temps"></a>

E.g. https://en.wikipedia.org/wiki/London

In [ ]:
req = requests.get('https://en.wikipedia.org/wiki/London')
soup = bs(req.content,'lxml')  
table = soup.find_all('table')
# also try: soup.find('table', {'class': 'wikitable sortable'}) 

df = pd.read_html(str(table))

table_raw = pd.DataFrame(df[1])
table_raw.head()

In [211]:
req = requests.get('https://en.wikipedia.org/wiki/Milan')
soup = bs(req.content,'lxml')  
table = soup.find_all('table')
# also try: soup.find('table', {'class': 'wikitable sortable'}) 

df = pd.read_html(str(table))

table_raw = pd.DataFrame(df[1])
table_raw.head()

Climate data for Milan (Linate Airport), elevation: 107 m or 351 ft, 1971–2000 normals, extremes 1946–present  \
                                                                                                          Month   
0                                Record high °C (°F)                                                              
1                               Average high °C (°F)                                                              
2                                 Daily mean °C (°F)                                                              
3                                Average low °C (°F)                                                              
4                                 Record low °C (°F)                                                              

                                                                           \
          Jan         Feb         Mar         Apr         May         Jun   
0  21.7(71.1)  23.8(74.8)  26.9(80.4)  32.4(90.3)  35.5(95.9)  36.6(97.9)   
1   5.9(42.6)   9.0(48.2)  14.3(57.7)  17.4(63.3)  22.3(72.1)  26.2(79.2)   
2   2.5(36.5)   4.7(40.5)   9.0(48.2)  12.2(54.0)  17.0(62.6)  20.8(69.4)   
3  −0.9(30.4)   0.3(32.5)   3.8(38.8)   7.0(44.6)  11.6(52.9)  15.4(59.7)   
4  −15.0(5.0)  −15.6(3.9)  −7.4(18.7)  −2.5(27.5)  −0.8(30.6)   5.6(42.1)   

                                                                           \
          Jul         Aug         Sep         Oct         Nov         Dec   
0  37.2(99.0)  36.9(98.4)  33.0(91.4)  28.2(82.8)  23.0(73.4)  21.2(70.2)   
1  29.2(84.6)  28.5(83.3)  24.4(75.9)  17.8(64.0)  10.7(51.3)   6.4(43.5)   
2  23.6(74.5)  23.0(73.4)  19.2(66.6)  13.4(56.1)   7.2(45.0)   3.3(37.9)   
3  18.0(64.4)  17.6(63.7)  14.0(57.2)   9.0(48.2)   3.7(38.7)   0.1(32.2)   
4   8.4(47.1)   8.0(46.4)   3.0(37.4)  −2.3(27.9)  −6.2(20.8)  −13.6(7.5)   

               
         Year  
0  37.2(99.0)  
1  17.7(63.9)  
2  13.0(55.4)  
3   8.3(46.9)  
4  −15.6(3.9)

In [212]:
table_raw.shape

(10, 14)

In [28]:
# table_raw.iloc[9,:]

In [213]:
table_raw.drop(labels=9, axis=0, inplace=True)

In [214]:
table_raw.shape

(9, 14)

In [215]:
table_raw

Climate data for Milan (Linate Airport), elevation: 107 m or 351 ft, 1971–2000 normals, extremes 1946–present  \
                                                                                                          Month   
0                                Record high °C (°F)                                                              
1                               Average high °C (°F)                                                              
2                                 Daily mean °C (°F)                                                              
3                                Average low °C (°F)                                                              
4                                 Record low °C (°F)                                                              
5                  Average precipitation mm (inches)                                                              
6              Average precipitation days (≥ 1.0 mm)                                                              
7                      Average relative humidity (%)                                                              
8                        Mean monthly sunshine hours                                                              

                                                                           \
          Jan         Feb         Mar         Apr         May         Jun   
0  21.7(71.1)  23.8(74.8)  26.9(80.4)  32.4(90.3)  35.5(95.9)  36.6(97.9)   
1   5.9(42.6)   9.0(48.2)  14.3(57.7)  17.4(63.3)  22.3(72.1)  26.2(79.2)   
2   2.5(36.5)   4.7(40.5)   9.0(48.2)  12.2(54.0)  17.0(62.6)  20.8(69.4)   
3  −0.9(30.4)   0.3(32.5)   3.8(38.8)   7.0(44.6)  11.6(52.9)  15.4(59.7)   
4  −15.0(5.0)  −15.6(3.9)  −7.4(18.7)  −2.5(27.5)  −0.8(30.6)   5.6(42.1)   
5  58.7(2.31)  49.2(1.94)  65.0(2.56)  75.5(2.97)  95.5(3.76)  66.7(2.63)   
6         6.7         5.3         6.7         8.1         8.9         7.7   
7          86          78          71          75          72          71   
8        58.9        96.1       151.9       177.0       210.8       243.0   

                                                                            \
          Jul         Aug         Sep          Oct         Nov         Dec   
0  37.2(99.0)  36.9(98.4)  33.0(91.4)   28.2(82.8)  23.0(73.4)  21.2(70.2)   
1  29.2(84.6)  28.5(83.3)  24.4(75.9)   17.8(64.0)  10.7(51.3)   6.4(43.5)   
2  23.6(74.5)  23.0(73.4)  19.2(66.6)   13.4(56.1)   7.2(45.0)   3.3(37.9)   
3  18.0(64.4)  17.6(63.7)  14.0(57.2)    9.0(48.2)   3.7(38.7)   0.1(32.2)   
4   8.4(47.1)   8.0(46.4)   3.0(37.4)   −2.3(27.9)  −6.2(20.8)  −13.6(7.5)   
5  66.8(2.63)  88.8(3.50)  93.1(3.67)  122.4(4.82)  76.7(3.02)  61.7(2.43)   
6         5.4         7.1         6.1          8.3         6.4         6.3   
7          71          72          74           81          85          86   
8       285.2       251.1       186.0        130.2        66.0        58.9   

                 
           Year  
0    37.2(99.0)  
1    17.7(63.9)  
2    13.0(55.4)  
3     8.3(46.9)  
4    −15.6(3.9)  
5  920.1(36.22)  
6          83.0  
7            77  
8        1915.1

In [217]:
# table_raw.dtypes

In [199]:
table_raw.iloc[0,0]

'Record high °C (°F)'

In [200]:
table_raw.iloc[0,0].replace(r"\s\(.*?\)","")   #.str.strip()

'Record high °C (°F)'

In [201]:
table_raw.iloc[0,0].strip('C')

'Record high °C (°F)'

In [202]:
table_raw.iloc[0,0].split(' (').pop(0)
# l.pop()
# type(l[0])
# float(l)
# float(l[0])

'Record high °C'

In [218]:
float(table_raw.iloc[0,1].split('(').pop(0))

21.7

In [228]:
table_raw.iloc[3,1]

'−0.9(30.4)'

In [229]:
table_raw.replace( table_raw.iloc[3,1],\
                  float(table_raw.iloc[3,1].split('(').pop(0)) ) #, inplace=True) 
table_raw.head(4)

ValueError: could not convert string to float: '−0.9'

In [222]:
for r in range(0,6):
    for c in range(1,14):
        table_raw.replace(table_raw.iloc[r,c], \
                          float(table_raw.iloc[r,c].split('(').pop(0)) ) #,inplace=True)

table_raw

ValueError: could not convert string to float: '−0.9'

In [230]:
float('-0.9')

-0.9

#### [back to contents](#contentz)

-------
--------

## Scraping a generic properties page <a name="prop-page"></a>

In [26]:
req = requests.get('https://search.savills.com/list?SearchList=Id_37507+Category_TownVillageCity&Tenure=GRS_T_B&SortOrder=SO_PCDD&MinPrice=500000&Currency=GBP&Period=Week&PropertyTypes=GRS_PT_APT&Bedrooms=GRS_B_2&ResidentialSizeUnit=SquareFeet&LandAreaUnit=Acre&SaleableAreaUnit=SquareFeet&AvailableSizeUnit=SquareFeet&Category=GRS_CAT_RES&LocationKey=englandlondonlondonec3n&Shapes=W3sidHlwZSI6IkNpcmNsZSIsImNvb3JkaW5hdGVzIjpbLTAuMTU0MzIyMDEwNDk4MDU1ODMsNTEuNDc0NjM5MjUzMjc5MTVdLCJyYWRpdXMiOiIxNjA5LjM0NG0iLCJsb2NhdGlvbklkIjoiMjgwMjcvODg5MTJlNGYtYmVmMi00MDQxLTgwY2MtYmU3M2RlOWNhNGE2In1d')
# print(req)  # 200=good

# soup = bs(req.content,'lxml')  
soup = bs(req.text, 'html.parser')
# table = soup.find('header', {'class':'sv-property-price__value'})

# df = pd.read_html(str(table))

# houses_raw = pd.DataFrame(df[1])
# houses_raw.head()

In [44]:
house_containers = soup.find_all('div', class_="sv-results-listing")
first = house_containers[0]

first.find_all('span')

[<span class="sv-slider__footer sv--bg"></span>,
 <span></span>,
 <span class="sv-slider__footer-action sv--paging">1/13</span>,
 <span class="sv-slider__footer-action sv--save-inline"><button class="sv-favourite" type="button"><span>Save</span></button></span>,
 <span>Save</span>,
 <span class="sv-details__address1--truncate" width="0"><span></span><span>Vardens Road</span><span style="position:fixed;visibility:hidden;top:0;left:0">…</span></span>,
 <span></span>,
 <span>Vardens Road</span>,
 <span style="position:fixed;visibility:hidden;top:0;left:0">…</span>,
 <span>680 sq ft</span>,
 <span>(63.17 sq m)</span>,
 <span class="sv-property-price__guide">Guide price</span>,
 <span class="sv-property-price__value"><span class="sv-property-price__rent-unit"></span><span>£700,000</span></span>,
 <span class="sv-property-price__rent-unit"></span>,
 <span>£700,000</span>,
 <span class="sv-view-lt-med" width="0"><span></span><span><span>Fantastic local amenities nearby</span></span><span styl

In [36]:
# first.find_all('span', class_='sv-property-price__rent-unit')

In [42]:
# size = 
first.find_all('span')[10].text.strip('()')

'63.17 sq m'

In [43]:
# price = 
first.find_all('span')[12].text

'£700,000'

In [51]:
first.find_all('span')[75].text.strip('()')

'79.34 sq m'

In [52]:
first.find_all('span')[77].text

'£650,000'

In [54]:
first.find_all('span')[75+65].text.strip('()')

'83.89 to 113.62 sq m'

In [60]:
first.find_all('span')[77+65].text.replace('\xa0',' ').replace('\u200b','')

'£1,660,000 - £1,800,000'

In [74]:
for i in range(0,6):
    print(first.find_all('span')[75+65*2+i].text.strip('()'))

…
976 sq ft
90.67 sq m
Guide price
£1,275,000



In [78]:
first.find_all('span')[75+65*2+2].text.strip('()')

'90.67 sq m'

In [84]:
first.find_all('span')[75+65*2+4].text

'£1,275,000'

In [83]:
first.find_all('span')[75+65*3+1].text.strip('()')

'126.35 sq m'

In [85]:
first.find_all('span')[75+65*3+3].text

'£1,225,000'

#### [back to contents](#contentz)

-------
--------

In [109]:
# testing other separators
hc = soup.find_all('ul', class_="sv-results-listing__inner")
first = hc[0]
first.find_all('span')[10].text.strip('()')
# first.find_all('span')[12].text

'63.17 sq m'

In [294]:
# testing other separators
hc2 = soup.find_all('li', class_="sv-results-listing__item")

metriq=[]
for i in range(0,10):
    
    try:
        m2 = hc2[i].find_all('span')[10].text.strip('()')
        metriq.apppend(m2)
    except:
        continue
        
metriq

# hc2[1].find_all('span')[12].text

[]

In [295]:
# building a loop for 'li'+'item' objects

hc2 = soup.find_all('li', class_="sv-results-listing__item")

props=[]
for i in range(0,50):
    
#     ssupp = hc2[i]
    
    try:        
        area = hc2[i].find_all('span')[10].text.strip('()')
        if ' sq m' in area:
            pass
        else:
            area = hc2[i].find_all('span')[11].text.strip('()')
    #     props.append(area)


        price = hc2[i].find_all('span')[12].text
        if '£' in price:
            pass
        else:
            price = hc2[i].find_all('span')[13].text
    #     props.append(price.replace('\xa0',' ').replace('\u200b',''))

    
#         pn = float(price.str.extract('(\d+)'))   #float(price.strip('£').replace(',',''))
#         an = float(area.str.extract('(\d+)'))    #float(area.strip(' sq m'))
        try:
            # if (pn.isnumeric() and an.isnumeric()):
            ppsqm = int( float(price.strip('£').replace(',','')) / float(area.strip(' sq m')))
#             props.append(ppsqm)
        except:
            ppsqm = 'N/A'
            
#         else:
#             pass


        props.append(( price.replace('\xa0',' ').replace('\u200b',''),\
                      area, f'{ppsqm:,}' ))

    except:
        continue
#         i+1 
        # print(f'Exception: Item {i}')
            
        
prop_data = pd.DataFrame(props, columns=['price','area','ppsqm'])
prop_data

# prop_data.to_excel('prop_data.xlsx')

,price,area,ppsqm
0,"£700,000",63.17 sq m,"11,081"
1,"£650,000",79.34 sq m,"8,192"
2,"£1,275,000",90.67 sq m,"14,061"
3,"£1,225,000",126.35 sq m,"9,695"
4,"£699,995",85.38 sq m,"8,198"
5,"£1,125,000",100.80 sq m,"11,160"
6,"£1,100,000",121.98 sq m,"9,017"
7,"£799,950",83.43 sq m,"9,588"
8,"£4,500,000",271.46 sq m,"16,577"
9,"£2,750,000",120.31 sq m,"22,857"


In [ ]:
# float(price.str.extract('(\d+)'))

In [297]:
# from itertools import takewhile
# float(''.join(takewhile(str.isdigit,area))) #--> missing the decimals!

In [ ]:
# ssupp = hc2[3]
# ssupp.find_all('span')[14].text.strip('()')

In [296]:
# hc2[1].find_all('span')[11].text.strip('()')

#### [back to contents](#contentz)

-------
--------

## Web scraper - property **prices** <a name="prop-price"></a>

In [179]:
# looping page numbers too

props=[]

for page in range(1,10+1):
    
    prop_url = 'https://search.savills.com/list?SearchList=Id_37507+Category_TownVillageCity&Tenure=GRS_T_B&SortOrder=SO_PCDD&MinPrice=500000&Currency=GBP&Period=Week&PropertyTypes=GRS_PT_APT&Bedrooms=GRS_B_2&ResidentialSizeUnit=SquareFeet&LandAreaUnit=Acre&SaleableAreaUnit=SquareFeet&AvailableSizeUnit=SquareFeet&Category=GRS_CAT_RES&LocationKey=englandlondonlondonec3n&Shapes=W3sidHlwZSI6IkNpcmNsZSIsImNvb3JkaW5hdGVzIjpbLTAuMTU0MzIyMDEwNDk4MDU1ODMsNTEuNDc0NjM5MjUzMjc5MTVdLCJyYWRpdXMiOiIxNjA5LjM0NG0iLCJsb2NhdGlvbklkIjoiMjgwMjcvODg5MTJlNGYtYmVmMi00MDQxLTgwY2MtYmU3M2RlOWNhNGE2In1d'+'&CurrentPage='+str(page)
    req = requests.get(prop_url) #, headers=headers)
    page_html = bs(req.text, 'html.parser')
    prop_items = page_html.find_all('li', class_="sv-results-listing__item")
    

    for i in range(0,20):

        try:        
            area = prop_items[i].find_all('span')[10].text.strip('()')
            if ' sq m' in area:
                pass
            else:
                area = prop_items[i].find_all('span')[11].text.strip('()')
#             props.append(area)


            price = prop_items[i].find_all('span')[12].text
            if '£' in price:
                pass
            else:
                price = prop_items[i].find_all('span')[13].text
#             props.append(price.replace('\xa0',' ').replace('\u200b',''))


   
            try:
                ppsqm = float(price.strip('£').replace(',','')) / float(area.strip(' sq m'))
#              props.append(ppsqm)
            except:
                ppsqm = 'N/A'

        
            props.append(( price.replace('\xa0',' ').replace('\u200b',''),\
                          area, int(ppsqm) ))  #f'{ppsqm:,}'

        except:
            continue

      
    
prop_price_data = pd.DataFrame(props, columns=['price','area','ppsqm'])
prop_price_data.head()



,price,area,ppsqm
0,"£850,000",86.40 sq m,9837
1,"£2,195,000",128.58 sq m,17071
2,"£1,795,000",77.95 sq m,23027
3,"£899,995",121.70 sq m,7395
4,"£700,000",52.03 sq m,13453
5,"£975,000",95.97 sq m,10159
6,"£2,000,000",136.29 sq m,14674
7,"£695,000",79.71 sq m,8719
8,"£735,000",69.40 sq m,10590
9,"£950,000",72.84 sq m,13042


In [180]:
prop_price_data.describe()

,ppsqm
count,146.000000
mean,12406.897260
std,5029.586165
min,4739.000000
25%,9328.500000
50%,10843.000000
75%,14409.000000
max,31725.000000


In [181]:
prop_price_data['ppsqm'].dropna().mean()

12406.897260273972

In [182]:
prop_price_data.dtypes

price    object
area     object
ppsqm     int64
dtype: object

In [183]:
# prop_price_data.to_excel('london_prop_price_data2.xlsx')

#### testing for rental pages

In [372]:
prop_url2 = 'https://search.savills.com/list?SearchList=Id_37507+Category_TownVillageCity&Tenure=GRS_T_R&SortOrder=SO_PCDD&Currency=GBP&Period=Month&PropertyTypes=GRS_PT_APT&Bedrooms=GRS_B_2&Bathrooms=-1&CarSpaces=-1&Receptions=-1&ResidentialSizeUnit=SquareFeet&LandAreaUnit=Acre&SaleableAreaUnit=SquareMeter&AvailableSizeUnit=SquareFeet&Category=GRS_CAT_RES&LocationKey=englandlondonlondonec3n&Shapes=W3sidHlwZSI6IkNpcmNsZSIsImNvb3JkaW5hdGVzIjpbLTAuMTU0MzIyMDEwNDk4MDU1ODMsNTEuNDc0NjM5MjUzMjc5MTVdLCJyYWRpdXMiOiIxNjA5bSIsImxvY2F0aW9uSWQiOiIyODAyNy84ODkxMmU0Zi1iZWYyLTQwNDEtODBjYy1iZTczZGU5Y2E0YTYifV0'
req2 = requests.get(prop_url2) #, headers=headers)
page_html2 = bs(req2.text, 'html.parser')
prop_items2 = page_html2.find_all('li', class_="sv-results-listing__item")

# prop_items2[0].find_all('span')
# prop_items2[0].find_all('span')[12].text
# prop_items2[0].find_all('span')[14].text

for i in range(0,15):
    print(5+i, prop_items2[0].find_all('span')[5+i].text.strip('()'))


5 Save
6 Save
7 Rathbone East, Canning Town…
8 
9 Rathbone East, Canning Town
10 …
11 785 sq ft
12 72.93 sq m
13 Guide price
14 £1,588
15 
16 £1,588
17 Monthly
18 No letting fees…
19 


#### [back to contents](#contentz)

-------
--------

## Web scraper - property **rentals** <a name="prop-rent"></a>

In [184]:

rents=[]

for page in range(1,10+1):
    
    prop_url2 = 'https://search.savills.com/list?SearchList=Id_37507+Category_TownVillageCity&Tenure=GRS_T_R&SortOrder=SO_PCDD&Currency=GBP&Period=Month&PropertyTypes=GRS_PT_APT&Bedrooms=GRS_B_2&Bathrooms=-1&CarSpaces=-1&Receptions=-1&ResidentialSizeUnit=SquareFeet&LandAreaUnit=Acre&SaleableAreaUnit=SquareMeter&AvailableSizeUnit=SquareFeet&Category=GRS_CAT_RES&LocationKey=englandlondonlondonec3n&Shapes=W3sidHlwZSI6IkNpcmNsZSIsImNvb3JkaW5hdGVzIjpbLTAuMTU0MzIyMDEwNDk4MDU1ODMsNTEuNDc0NjM5MjUzMjc5MTVdLCJyYWRpdXMiOiIxNjA5bSIsImxvY2F0aW9uSWQiOiIyODAyNy84ODkxMmU0Zi1iZWYyLTQwNDEtODBjYy1iZTczZGU5Y2E0YTYifV0'+'&CurrentPage='+str(page)
    req2 = requests.get(prop_url2) #, headers=headers)
    page_html2 = bs(req2.text, 'html.parser')
    prop_items2 = page_html2.find_all('li', class_="sv-results-listing__item")
    

    for i in range(0,20):

        try:        
            area = prop_items2[i].find_all('span')[11].text.strip('()')
            if ' sq m' in area:
                pass
            else:
                area = prop_items2[i].find_all('span')[12].text.strip('()')
#             props.append(area)


            rentpm = prop_items2[i].find_all('span')[13].text
            if '£' in rentpm:
                pass
            else:
                rentpm = prop_items2[i].find_all('span')[14].text
#             props.append(price.replace('\xa0',' ').replace('\u200b',''))


   
            try:
                rpsqm = float(rentpm.strip('£').replace(',',''))*12 / float(area.strip(' sq m'))
#              props.append(ppsqm)
            except:
                rpsqm = 'N/A'

        
            rents.append(( rentpm.replace('\xa0',' ').replace('\u200b',''),\
                          area, int(rpsqm) ))  #f'{ppsqm:,}'

        except:
            continue

      
    
prop_rent_data = pd.DataFrame(rents, columns=['rentpm','area','rpsqm(pa)'])
prop_rent_data.head()


,rentpm,area,rpsqm(pa)
0,"£5,417",161.19 sq m,403
1,"£2,383",86.77 sq m,329
2,"£2,360",86.40 sq m,327
3,"£3,358",113.43 sq m,355
4,"£1,550",87.42 sq m,212


In [185]:
prop_rent_data.describe()

,rpsqm(pa)
count,137.000000
mean,417.591241
std,134.707720
min,210.000000
25%,329.000000
50%,392.000000
75%,486.000000
max,1232.000000


In [187]:
prop_rent_data['rpsqm(pa)'].dropna().mean()

417.59124087591243

In [188]:
avg_rent_yield = round(prop_rent_data['rpsqm(pa)'].mean() /\
                       prop_price_data['ppsqm'].mean() *100 ,2)
print(f'Rent Yield avg = {avg_rent_yield:,}%') 
# correct yield avg, as takes into account prop size

Rent Yield avg = 3.37%


In [252]:
# prop_rent_data.to_excel('london_prop_rent_data2.xlsx')

#### [back to contents](#contentz)

-------
--------

#### Extract average property price and rental pm [FYI only]

In [228]:
import numpy as np
import pandas as pd
import warnings; warnings.filterwarnings("ignore")
prices = prop_price_data
# pd.read_excel('london_prop_price_data1.xlsx' , index_col=0)
# prices.head()
rents = prop_rent_data
# pd.read_excel('london_prop_rent_data1.xlsx' , index_col=0)
# rents.head()

In [233]:
# rents['rentpm'].strip('£').replace(',','')
# type(rents['rentpm'][0])

# int( rents['rentpm'][0].strip('£').replace(',','') )
# len(rents['rentpm'][0])

warnings.filterwarnings("ignore")
                        
for r in range(0,len(rents['rentpm'])):
    rents['rentpm'][r] = rents['rentpm'][r].strip('£').replace(',','')

# rents['rentpm'][0:3]  
rent_avg = int(rents['rentpm'].astype(int).mean())
rent_std = int(rents['rentpm'].astype(int).std())
print(f' Rent avg = £{rent_avg:,} , +-stdev £{rent_std:,} \
--> (£{rent_avg-rent_std:,} <> £{rent_avg+rent_std:,})') 


for p in range(0,len(prices['price'])):
    prices['price'][p] = prices['price'][p].strip('£').replace(',','')

# rents['rentpm'][0:3]  
price_avg = int(prices['price'].astype(int).mean())
price_std = int(prices['price'].astype(int).std())
print(f' Price avg = £{price_avg:,} , +-stdev £{price_std:,} \
--> (£{price_avg-price_std:,} <> £{price_avg+price_std:,})') 

# !!ignore this!!
# !!avg size of prop *for sale* may well differ avg size of prop *for rent*!!
# avg_rent_yield2 = round(rent_avg *12 / price_avg *100 ,2)
# print(f'Rent Yield avg = {avg_rent_yield2:,}%')

warnings.filterwarnings("default")


 Rent avg = £3,211 , +-stdev £1,507 --> (£1,704 <> £4,718)
 Price avg = £1,368,316 , +-stdev £1,106,300 --> (£262,016 <> £2,474,616)


In [ ]:
# float(prop_rent_data['rentpm'][0].strip('£').replace(',',''))

In [249]:
# prices['price'].astype(int).std()
# prices['price'].astype(int).values.std()
# rents['rentpm'].astype(int).values.mean()

In [414]:
# prop_items2[0].find_all('span')[12].text

In [415]:
# prop_items2[1].find_all('span')[11].text

In [416]:
# prop_items2[0].find_all('span')[14].text

In [417]:
# prop_items2[1].find_all('span')[13].text

#### [back to contents](#contentz)

--------
---------

### Scraping for city indices - Numbeo <a name="numb"></a>

#### Quality of life <a name="qual-lif"></a>

https://www.numbeo.com/quality-of-life/rankings_current.jsp

In [270]:
req = requests.get('https://www.numbeo.com/quality-of-life/rankings_current.jsp')
soup = bs(req.content,'lxml')  
table = soup.find_all('table')
# also try: soup.find('table', {'class': 'wikitable sortable'}) 

df = pd.read_html(str(table))

qual_life = pd.DataFrame(df[2])
qual_life.head()

,Rank,City,Quality of Life Index,Purchasing Power Index,Safety Index,Health Care Index,Cost of Living Index,Property Price to Income Ratio,Traffic Commute Time Index,Pollution Index,Climate Index
0,NaN,"Adelaide, Australia",207.83,108.87,71.37,80.91,71.20,4.27,24.13,17.94,94.96
1,NaN,"Canberra, Australia",206.65,103.63,78.93,81.44,78.38,5.21,24.05,14.01,82.72
2,NaN,"Raleigh, NC, United States",202.53,129.80,66.17,75.62,70.25,2.82,32.41,21.95,83.88
3,NaN,"Wellington, New Zealand",200.21,97.75,70.07,74.90,71.02,6.36,27.74,13.66,97.68
4,NaN,"Madison, WI, United States",197.34,119.50,68.89,78.89,67.15,3.43,23.73,17.51,51.64


In [266]:
# qual_life.to_excel('quality_life_cities_25-07-20.xlsx')

#### [back to contents](#contentz)

-------
--------

#### Cost of living <a name="cost-liv"></a>

https://www.numbeo.com/cost-of-living/rankings_current.jsp

In [262]:
req = requests.get('https://www.numbeo.com/cost-of-living/rankings_current.jsp')
soup = bs(req.content,'lxml')  
table = soup.find_all('table')
# also try: soup.find('table', {'class': 'wikitable sortable'}) 

df = pd.read_html(str(table))

cost_liv = pd.DataFrame(df[2])
cost_liv.head()

,Rank,City,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
0,NaN,"Zurich, Switzerland",136.91,65.72,102.15,139.13,121.60,118.14
1,NaN,"Lugano, Switzerland",131.93,40.35,87.22,138.06,114.20,98.66
2,NaN,"Basel, Switzerland",131.82,46.78,90.30,129.10,130.72,107.20
3,NaN,"Geneva, Switzerland",127.65,67.68,98.37,126.36,119.96,105.88
4,NaN,"Lausanne, Switzerland",126.05,52.15,89.97,128.51,116.97,108.20


In [263]:
# cost_liv.to_excel('cost_living_cities_25-07-20.xlsx')

#### [back to contents](#contentz)

----
----

#### Property stats <a name="prop-stats"></a>

https://www.numbeo.com/property-investment/rankings_current.jsp

In [271]:
req = requests.get('https://www.numbeo.com/property-investment/rankings_current.jsp')
soup = bs(req.content,'lxml')  
table = soup.find_all('table')
# also try: soup.find('table', {'class': 'wikitable sortable'}) 

df = pd.read_html(str(table))

prop_stats = pd.DataFrame(df[2])
prop_stats.head()

,Rank,City,Price To Income Ratio,Gross Rental Yield City Centre,Gross Rental Yield Outside of Centre,Price To Rent Ratio City Centre,Price To Rent Ratio Outside Of City Centre,Mortgage As A Percentage Of Income,Affordability Index
0,NaN,"Damascus, Syria",70.71,3.79,3.57,26.36,27.98,827.05,0.12
1,NaN,"Shenzhen, China",46.74,1.33,1.39,74.99,71.94,375.73,0.27
2,NaN,"Hong Kong, Hong Kong",43.40,1.72,1.75,57.99,57.05,277.83,0.36
3,NaN,"Tehran, Iran",42.66,4.57,5.33,21.89,18.76,895.35,0.11
4,NaN,"Beijing, China",41.56,1.66,2.12,60.19,47.25,335.64,0.30


In [272]:
# prop_stats.to_excel('prop_stats_cities_25-07-20.xlsx')

#### [back to contents](#contentz)

-------
---------

### Importing consolidated Numbeo data <a name="import-numb"></a>

In [6]:
pd.set_option('display.max_columns', None)
city_info_all = pd.read_excel('geo_locs_with_coords_and_info.xlsx')
#                               ,index_col=0)
# city_info.reset_index(drop=True, inplace=True)
city_info_all.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,2,Unnamed: 7,Unnamed: 8,7,6,3,Unnamed: 12,Unnamed: 13,8,9,Unnamed: 16,Unnamed: 17,4,5,Unnamed: 20,Unnamed: 21,1b,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,10,1,Unnamed: 29,Unnamed: 30
0,NaN,City,Country,Latitude,Longitude,City+,Quality of Life Index,Purchasing Power Index,Safety Index,Crime Index,Health Care Index,Cost of Living Index,Property Price to Income Ratio,Traffic Commute Time Index,Pollution Index,Climate (Weather) Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,Price To Income Ratio,Gross Rental Yield City Centre,Gross Rental Yield Outside of Centre,Price To Rent Ratio City Centre,Price To Rent Ratio Outside Of City Centre,Mortgage As A Percentage Of Income,Affordability Index,Price in $ per sqm - Apartment in City Centre,NaN,75.000000
1,0.0,London,UK,51.5073,-0.127647,"London, United Kingdom",128.23,82.03,47.29,52.71,70.28,81.1,15.97,45.26,58.57,88.25,66.77,74.1,59.12,82.82,82.03,15.97,3.34,4.21,29.92,23.75,108.07,0.93,14284,NaN,1.071300
2,1.0,Milan,Italy,45.4668,9.1905,"Milan, Italy",117.27,51.73,56.81,43.19,71.57,82.37,19.11,36.88,66.06,88.12,43.78,63.53,71.36,79.45,51.47,19.11,3.16,4.56,31.66,21.91,115.33,0.87,10214,NaN,0.766050
3,2.0,Paris,France,48.8567,2.35146,"Paris, France",117.69,68.81,48.15,51.85,78.58,91.01,21.8,42.17,64.23,88.39,46.63,69.34,89.96,77.63,68.37,21.8,2.36,2.2,42.34,45.39,124.75,0.8,14105,NaN,1.057875
4,3.0,Frankfurt,Germany,50.1106,8.68209,"Frankfurt, Germany",171.91,97.83,58.26,41.74,74.51,73.86,9.49,25.11,37.45,84.72,39.47,57.06,52.76,66.52,97.2,9.49,3.68,4.28,27.19,23.34,56.95,1.76,NaN,NaN,NaN


#### Cities selection

In [20]:
chosen10 = ['London, United Kingdom','Milan, Italy','Paris, France','Zurich, Switzerland',
'Tokyo, Japan','Sidney, Australia','Singapore, Singapore','Abu Dhabi, United Arab Emirates',
'New York, NY, United States','San Francisco, CA, United States']
city_info = city_info_all.loc[ city_info_all['City+'].isin(chosen10) ]
city_info.reset_index(drop=True, inplace=True)
city_info

,City,Nation,Latitude,Longitude,City+,Quality of Life Index,Purchasing Power Index,Safety Index,Health Care Index,Cost of Living Index,Property Price to Income Ratio,Traffic Commute Time Index,Pollution Index,Climate Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,Price To Income Ratio,Gross Rental Yield City Centre,Gross Rental Yield Outside of Centre,Price To Rent Ratio City Centre,Price To Rent Ratio Outside Of City Centre,Mortgage As A Percentage Of Income,Affordability Index
0,London,UK,51.507322,-0.127647,"London, United Kingdom",128.23,82.03,47.29,70.28,81.10,15.97,45.26,58.57,88.25,66.77,74.10,59.12,82.82,82.03,15.97,3.34,4.21,29.92,23.75,108.07,0.93
1,Milan,Italy,45.466800,9.190500,"Milan, Italy",117.27,51.73,56.81,71.57,82.37,19.11,36.88,66.06,88.12,43.78,63.53,71.36,79.45,51.47,19.11,3.16,4.56,31.66,21.91,115.33,0.87
2,Paris,France,48.856697,2.351462,"Paris, France",117.69,68.81,48.15,78.58,91.01,21.80,42.17,64.23,88.39,46.63,69.34,89.96,77.63,68.37,21.80,2.36,2.20,42.34,45.39,124.75,0.80
3,Zurich,Switzerland,47.372394,8.542333,"Zurich, Switzerland",196.06,118.14,83.44,74.58,136.91,7.91,33.72,17.17,81.48,65.72,102.15,139.13,121.60,118.14,7.91,3.43,3.92,29.15,25.50,45.10,2.22
4,Tokyo,Japan,35.682839,139.759455,"Tokyo, Japan",153.26,71.51,76.60,80.13,91.75,16.53,40.49,42.25,85.26,40.62,66.78,94.46,49.48,71.51,16.53,2.65,2.49,37.70,40.21,95.08,1.05
5,Sidney,Australia,-33.854816,151.216454,"Sidney, Australia",175.79,93.44,66.68,77.75,84.63,10.33,43.40,26.82,97.07,55.45,70.38,77.99,71.35,93.44,10.33,4.07,4.79,24.59,20.88,72.09,1.39
6,Singapore,Singapore,1.357107,103.819499,"Singapore, Singapore",140.47,80.42,67.90,71.07,82.51,22.17,41.25,33.26,57.45,61.76,72.38,71.56,57.91,80.42,22.17,2.35,2.89,42.50,34.61,140.19,0.71
7,Abu Dhabi,UAE,24.474796,54.370576,"Abu Dhabi, United Arab Emirates",166.94,88.02,88.51,71.56,61.77,5.84,26.51,45.76,43.89,41.82,52.03,51.40,58.40,88.01,5.84,8.07,8.13,12.39,12.31,42.57,2.35
8,New York,United States,40.712728,-74.006015,"New York, NY, United States",139.69,100.00,54.86,62.96,100.00,9.88,43.21,57.00,79.66,100.00,100.00,100.00,100.00,100.00,9.88,5.10,5.52,19.61,18.11,72.06,1.39
9,San Francisco,United States,37.779026,-122.419906,"San Francisco, CA, United States",161.12,135.51,44.70,66.02,92.68,7.33,52.74,46.95,97.26,109.64,100.96,91.85,87.01,135.51,7.33,5.70,5.90,17.54,16.95,52.83,1.89


In [7]:
# loading final summary file
cities_info = pd.read_excel('geo_locs_with_coords_and_info_FINAL.xlsx', index_col=0).iloc[:,:15]
p = 'Price per sqm - Apartment in City Centre'
cities_info[p] = cities_info.apply(lambda x: '${:,.0f}'.format(x[p]), axis=1)
cities_info.drop(columns=['Latitude','Longitude'],inplace=True)
cities_info.index = cities_info.index+1
cities_info 

,City,Country,Price per sqm - Apartment in City Centre,Quality of Life Index,Cost of Living Index,Groceries Index,Restaurant Price Index,Health Care Index,Crime Index,Pollution Index,Climate (Weather) Index,Gross Rental Yield City Centre,Affordability Index
1,Dubai,UAE,"$3,653",154.03,68.83,53.87,65.63,67.34,16.70,50.65,50.27,10.10,2.62
2,London,UK,"$14,284",128.23,81.10,59.12,82.82,70.28,52.71,58.57,88.25,3.34,0.93
3,Milan,Italy,"$10,214",117.27,82.37,71.36,79.45,71.57,43.19,66.06,88.12,3.16,0.87
4,New York,United States,"$15,685",139.69,100.00,100.00,100.00,62.96,45.14,57.00,79.66,5.10,1.39
5,Paris,France,"$14,105",117.69,91.01,89.96,77.63,78.58,51.85,64.23,88.39,2.36,0.80
6,San Francisco,United States,"$13,489",161.12,92.68,91.85,87.01,66.02,55.30,46.95,97.26,5.70,1.89
7,Singapore,Singapore,"$20,086",140.47,82.51,71.56,57.91,71.07,32.10,33.26,57.45,2.35,0.71
8,Sydney,Australia,"$10,136",175.79,84.63,77.99,71.35,77.75,33.32,26.82,97.07,4.07,1.39
9,Tokyo,Japan,"$11,274",153.26,91.75,94.46,49.48,80.13,23.40,42.25,85.26,2.65,1.05
10,Zurich,Switzerland,"$13,639",196.06,136.91,139.13,121.60,74.58,16.56,17.17,81.48,3.43,2.22


In [147]:
# save to file
# cities_info.to_csv('cities_info_table_final.csv')

In [181]:
# pd.set_option('display.max_colwidth', 1000)
# pd.set_option('max_colwidth', None)
# pd.options.display.width = None
# pd.options.display.width = None
# pd.set_option('display.max_rowheight', 100)
# cities_info.columns[0].width = 100
# pd.get_option('display.max_colwidth')
# cities_info

In [170]:
# pd.describe_option()

#### [back to contents](#contentz)

-------
---------

### Mapping a city index with variable color markers <a name="colors"></a>

Quality of Life

In [11]:
# centre the map
latitude_0 = 0.0  #location_0.latitude
longitude_0 = 0.0  #location_0.longitude
map_colors = folium.Map(location=[latitude_0, longitude_0], zoom_start=1)

# title = 'World Cities'
title_html = '''
             <h3 align="center" style="font-size:20px"><b>World Cities - Quality of Life</b></h3>
             '''
map_colors.get_root().html.add_child(folium.Element(title_html))

### select the index to include in mapping ###
city_index = cities_info['Quality of Life Index'] #['Cost of Living Index']   

# ['red','yellow','green'].reverse() if city_index in [ locs_info['Cost of Living Index']
color_list = ['red','yellow','green'] #[::-1]
# .reverse() or [::-1]
              
linear = bcm.LinearColormap( color_list,vmin=city_index.min(),
                            vmax=city_index.max() )

for lat, lon, city, index_value in zip( cities_info['Latitude'], cities_info['Longitude'],
                                       cities_info['City'], city_index ):

    label = '{}, {}'.format(city, index_value)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(  #Marker --> only allows predefined set of colours
        location=[lat, lon],
        radius=7.5,
        popup=label,
#         icon=folium.Icon(color='white' , icon='home', icon_color=linear(index_value) ) 
        color='black',  #rainbow[ind], #'darkred',
        weight=1.5,
        fill=True,
        fill_color=linear(index_value), #'yellow',  #'#3186cc',
        fill_opacity=1,
#         parse_html=False
    ).add_to(map_colors)

map_colors

Cost of Living

In [17]:
# centre the map
latitude_0 = 0.0  #location_0.latitude
longitude_0 = 0.0  #location_0.longitude
map_colors2 = folium.Map(location=[latitude_0, longitude_0], zoom_start=1)

# title = 'World Cities'
title_html = '''
             <h3 align="center" style="font-size:20px"><b>World Cities - Cost of Living</b></h3>
             '''
map_colors2.get_root().html.add_child(folium.Element(title_html))

### select the index to include in mapping ###
city_index = city_info['Cost of Living Index'] 

# ['red','yellow','green'].reverse() if city_index in [ locs_info['Cost of Living Index']
color_list = ['red','yellow','green'][::-1]
# .reverse() or [::-1]
              
linear = bcm.LinearColormap( color_list,vmin=city_index.min(),
                            vmax=city_index.max() )

for lat, lon, city, index_value in zip( city_info['Latitude'], city_info['Longitude'],
                                       city_info['City'], city_index ):

    label = '{}, {}'.format(city, index_value)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(  #Marker --> only allows predefined set of colours
        location=[lat, lon],
        radius=7.5,
        popup=label,
#         icon=folium.Icon(color='white' , icon='home', icon_color=linear(index_value) ) 
        color='black',  #rainbow[ind], #'darkred',
        weight=1.5,
        fill=True,
        fill_color=linear(index_value), #'yellow',  #'#3186cc',
        fill_opacity=1,
#         parse_html=False
    ).add_to(map_colors2)

map_colors2

#### [back to contents](#contentz)

-------
---------